In [28]:
import json
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

import re
import string

In [43]:
path = '/Users/noahedelstein/Desktop/Parsons/design with ml/ml_final/ml_final_python/state-of-the-union/transcripts.json'

with open(path) as data_file:
  data = data_file.read()
  # print(data) 
  data_content = json.loads(data)
  president = [line['president'] for line in data_content]
  dates = [line['date'] for line in data_content]
#   print(date)
#   transcripts = [line['transcript'] for line in data_content]
#   print(transcripts[0])

In [30]:
stop_words = set(stopwords.words('english'))


In [31]:
def tokenize(text):
    # first change all to lower case
    text = text.lower()
    # remove the http urls with a regular expression
    remove_urls = re.sub(r"http\S+", "", text)
    # remove punctuation - small note - periods are not important but other punctuation
    # could be useful to leave in!
    remove_punctuation = remove_urls.translate(str.maketrans('','',string.punctuation))
    # tokenize our sentences into words
    tokens = nltk.word_tokenize(remove_punctuation)
    remove_stops = [w for w in tokens if not w in stop_words]
    stems = [PorterStemmer().stem(t) for t in remove_stops]
    return stems

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

# calculating the tf-idf may take a while
print("calculating tf-idf")
tfidf = TfidfVectorizer(tokenizer=tokenize)
tfs = tfidf.fit_transform(transcripts)

calculating tf-idf


In [57]:
from sklearn.decomposition import TruncatedSVD

print("reducing tf-idf to 2 dim")
tfs_reduced = TruncatedSVD(n_components=2, random_state=0).fit_transform(tfs)
print("done")

reducing tf-idf to 2 dim
done


In [58]:
print(tfs_reduced)

[[ 4.04167602e-01  3.58701260e-01]
 [ 4.60119675e-01  3.72034618e-01]
 [ 4.09314427e-01  4.36664316e-01]
 [ 4.42697662e-01  4.74247617e-01]
 [ 4.55825587e-01  5.01986131e-01]
 [ 4.78979969e-01  4.92122906e-01]
 [ 4.45095102e-01  4.64507266e-01]
 [ 4.60952834e-01  4.83752442e-01]
 [ 4.87272296e-01  4.87636515e-01]
 [ 4.85579141e-01  4.97330065e-01]
 [ 4.62702799e-01  4.25640812e-01]
 [ 4.43315228e-01  3.69291334e-01]
 [ 5.07086453e-01  4.51361366e-01]
 [ 4.81421158e-01  3.92114963e-01]
 [ 4.59603047e-01  4.33163198e-01]
 [ 4.09091149e-01  3.46554126e-01]
 [ 4.07127933e-01  3.68710018e-01]
 [ 4.81652279e-01  4.65850141e-01]
 [ 4.89700258e-01  5.27800543e-01]
 [ 5.04340523e-01  4.97579414e-01]
 [ 5.18171131e-01  5.23660146e-01]
 [ 4.95184220e-01  5.16845948e-01]
 [ 4.79576580e-01  4.83032826e-01]
 [ 5.35319223e-01  4.84478696e-01]
 [ 5.04148965e-01  4.95593265e-01]
 [ 5.44995803e-01  5.21197805e-01]
 [ 4.81227717e-01  4.70017209e-01]
 [ 4.92399189e-01  4.36807798e-01]
 [ 4.53870558e-01  4

In [53]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
scaled = ss.fit_transform(tfs_reduced)

In [54]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=10)
clusters = kmeans.fit_predict(scaled)

In [55]:
from sklearn.neighbors import NearestNeighbors

neighbors = NearestNeighbors(n_neighbors=5, metric='cosine').fit(scaled)
_, closest = neighbors.kneighbors(kmeans.cluster_centers_)

In [56]:
for index, cluster_row in enumerate(closest):
    print('\n\nCluster number ' + str(index) + '\n\n')
    for col in cluster_row:
        print(dates[col], president[col])
        print('\n\n')



Cluster number 0


1877-12-03 Rutherford B. Hayes



1869-12-06 Ulysses S. Grant



1903-12-07 Theodore Roosevelt



1888-12-03 Grover Cleveland



1830-12-06 Andrew Jackson





Cluster number 1


1978-01-19 Jimmy Carter



1998-01-27 William J. Clinton



1971-01-22 Richard Nixon



1968-01-17 Lyndon B. Johnson



1965-01-04 Lyndon B. Johnson





Cluster number 2


1793-12-03 George Washington



1796-12-07 George Washington



1895-12-02 Grover Cleveland



1818-11-16 James Monroe



1819-12-07 James Monroe





Cluster number 3


1927-12-06 Calvin Coolidge



1906-12-03 Theodore Roosevelt



1907-12-03 Theodore Roosevelt



1905-12-05 Theodore Roosevelt



1904-12-06 Theodore Roosevelt





Cluster number 4


2014-01-28 Barack Obama



2011-01-25 Barack Obama



1988-01-25 Ronald Reagan



2001-02-27 George W. Bush



1992-01-28 George Bush





Cluster number 5


1936-01-03 Franklin D. Roosevelt



1945-01-06 Franklin D. Roosevelt



1944-01-11 Franklin D. Roosevelt



1956-01-

In [ ]:
lookup = []

with open(path) as data_file:
    data = data_file.read()
    data_content = json.loads(data)
    transcripts = [line['transcript'] for line in data_content]
    dates = [line['date'] for line in data_content]
    presidents = [line['president'] for line in data_content]
    for date, president, cluster_pos, transcript in zip(dates, presidents, tfs_reduced, transcripts):
        lookup.append({
            "Date": date,
            "President": president,
            "Cluster_Pos": cluster_pos.tolist(),
            "Speech": transcript
        })